# Customizing and controlling xclim

xclim's behaviour can be controlled globally or contextually through `xclim.set_options`, which acts the same way as `xarray.set_options`. For the extension of xclim with the addition of indicators, see the [Extending xclim](extendxclim.ipynb) notebook.

In [ ]:
from __future__ import annotations

import xarray as xr

import xclim

Let's create fake data with some missing values and mask every 10th, 20th and 30th of the month. This represents 9.6-10% of masked data for all months except February, where it is 7.1%.

In [ ]:
tasmax = xr.tutorial.load_dataset("air_temperature").air.resample(time="D").max(keep_attrs=True)
tasmax = tasmax.where(tasmax.time.dt.day % 10 != 0)

## Checks
Above, we created fake temperature data from a xarray tutorial dataset that doesn't have all the standard CF attributes. By default, when triggering a computation with an Indicator from xclim, warnings will be raised:

In [ ]:
tx_mean = xclim.atmos.tx_mean(tasmax=tasmax, freq="MS")  # compute monthly max tasmax

Setting `cf_compliance` to `'log'` mutes those warnings and sends them to the log instead.

In [ ]:
xclim.set_options(cf_compliance="log")

tx_mean = xclim.atmos.tx_mean(tasmax=tasmax, freq="MS")  # compute monthly max tasmax

## Adding translated metadata

With the help of its internationalization module (`xclim.core.locales`), xclim can add translated metadata to the output of the indicators. The metadata is _not_ translated on-the-fly, but translations are manually written for each indicator and metadata field. Currently, all indicators have a French translation, but users can freely add more languages. See [Internationalization](../internationalization.rst) and [Extending xclim](extendxclim.ipynb).

In the example below, notice the added `long_name_fr` and `description_fr` attributes. Also, the use of `set_options` as a context makes this configuration transient, only valid within the context.

In [ ]:
with xclim.set_options(metadata_locales=["fr"]):
    out = xclim.atmos.tx_max(tasmax=tasmax)
out.attrs

## Missing values

One can also globally change the missing method.

Change the default missing method to "pct" and set its tolerance to 8%:

In [ ]:
xclim.set_options(check_missing="pct", missing_options={"pct": {"tolerance": 0.08}})

tx_mean = xclim.atmos.tx_mean(tasmax=tasmax, freq="MS")  # compute monthly max tasmax
tx_mean.sel(time="2013", lat=75, lon=200)

Only February has non-masked data. Let's say we want to use the ``"wmo"`` method (and its default options), but only once, we can do:

In [ ]:
with xclim.set_options(check_missing="wmo"):
    tx_mean = xclim.atmos.tx_mean(tasmax=tasmax, freq="MS")  # compute monthly max tasmax
tx_mean.sel(time="2013", lat=75, lon=200)

This method checks that there are less than `nm=11` invalid values in a month and that there are no consecutive runs of `nc>=5` invalid values. Thus, every month is now valid.

<div class="alert alert-warning">

The content that follows is based on an experimental part of xclim, the way missing methods are implemented might change in the near future. Access of missing methods as functions of ``xclim.core.missing`` and usage of these algorithms in the indicators will be preserved, but custom subclasses might break with future changes.

</div>

Finally, it is possible for advanced users to register their own methods. Xclim's missing methods are in fact class-based. To create a custom missing class, one should implement a subclass of `xclim.core.checks.MissingBase` and override at least the `is_missing` method. This method should take the following arguments:

- `valid`, a `DataArray` of the mask of valid values in the input data array (with the same time coordinate as the raw data).
- `count`, `DataArray` of the number of days in each resampled periods
- `freq`, the resampling frequency.

The `is_missing` method should return a boolean mask, resampled at the `freq` frequency, the same as the indicator output (same as `count`), where `True` values are for elements that are considered missing and masked on the output.

To add additional arguments, one should override the `__init__` (receiving those arguments) and the `validate` static method, which validates them. The options are then stored in the `options` property of the instance. See example below and the docstrings in the module.

When registering the class with the `xclim.core.checks.register_missing_method` decorator, the keyword arguments will be registered as options for the missing method.

In [ ]:
from xclim.core.missing import MissingBase, register_missing_method
from xclim.indices.run_length import longest_run


@register_missing_method("consecutive")
class MissingConsecutive(MissingBase):
    """Any period with more than max_n consecutive missing values is considered invalid"""

    def __init__(self, max_n: int = 5):
        super().__init__(max_n=max_n)

    def is_missing(self, valid, count, freq):
        """Return a boolean mask for elements that are considered missing and masked on the output."""
        null = ~valid
        return null.resample(time=freq).map(longest_run, dim="time") >= self.options["max_n"]

    @staticmethod
    def validate(max_n):
        """Return whether the options are valid."""
        return max_n > 0

The new method is now accessible and usable with:

In [ ]:
with xclim.set_options(check_missing="consecutive", missing_options={"consecutive": {"max_n": 2}}):
    tx_mean = xclim.atmos.tx_mean(tasmax=tasmax, freq="MS")  # compute monthly max tasmax
tx_mean.sel(time="2013", lat=75, lon=200)